In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import os.path as path
import itertools
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, \
                                        EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical, multi_gpu_model, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [4]:
data_dir = 'data/'

In [ ]:
train_data = np.load(path.join(data_dir, 'imagenet_6_class_172_train_data.npz'))
val_data = np.load(path.join(data_dir, 'imagenet_6_class_172_val_data.npz'))

In [ ]:
x_train = train_data['x_data']
y_train = train_data['y_data']
x_val = val_data['x_data']
y_val = val_data['y_data']
x_test = x_val
y_test = y_val
y_list = val_data['y_list']

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape, y_list.shape

((235111, 172, 172, 3),
 (235111,),
 (3183, 172, 172, 3),
 (3183,),
 (3183, 172, 172, 3),
 (3183,),
 (6,))

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = y_val

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((235111, 172, 172, 3),
 (235111, 6),
 (3183, 172, 172, 3),
 (3183, 6),
 (3183, 172, 172, 3),
 (3183, 6))

In [ ]:
input_shape = x_train[0].shape
output_size = len(y_list)

In [ ]:
def build_2d_cnn_custom_ch_64_DO(conv_num=1):
    input_layer = Input(shape=input_shape)
    x = input_layer

    for i in range(conv_num):
        x = Conv2D(kernel_size=5, filters=64*(2**(i//2)), strides=(1,1), padding='same')(x)
#         x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=3, strides=(2,2), padding='same')(x)

    x = Flatten()(x)
  
    x = Dropout(0.75)(x)
    output_layer = Dense(output_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [ ]:
for i in range(1, 8):
    model = build_2d_cnn_custom_ch_64_DO(conv_num=i)
    model.summary()
    del model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 172, 172, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 172, 172, 64)      4864      
_________________________________________________________________
activation (Activation)      (None, 172, 172, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 86, 86, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 473344)            0         
_________________________________________________________________
dropout (Dropout)    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 172, 172, 3)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 172, 172, 64)      4864      
_________________________________________________________________
activation_15 (Activation)   (None, 172, 172, 64)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 86, 86, 64)        102464    
_________________________________________________________________
activation_16 (Activation)   (None, 86, 86, 64)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 43, 43, 64)        0         
__________

In [ ]:
class BalanceDataGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size, shuffle=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sample_size = int(np.sum(y_data, axis=0).min())
        self.data_shape = x_data.shape[1:]
        self.y_label = self.y_data.argmax(axis=1)
        self.labels = np.unique(self.y_label)
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.labels) * self.sample_size / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.zeros((len(self.labels), self.sample_size))
        for i, label in enumerate(self.labels):
            y_index = np.argwhere(self.y_label==label).squeeze()
            if self.shuffle == True:
                self.indexes[i] = np.random.choice(y_index, 
                                   self.sample_size, 
                                   replace=False)
            else:
                self.indexes[i] = y_index[:self.sample_size]
                
        self.indexes = self.indexes.flatten().astype(np.int32)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
                
    def __getitem__(self, batch_idx):
        indices = self.indexes[batch_idx*self.batch_size: (batch_idx+1)*self.batch_size]
        return self.x_data[indices], self.y_data[indices]    

In [ ]:
batch_size = 40
data_generator = BalanceDataGenerator(x_train, y_train,
                                      batch_size=batch_size)

In [ ]:
for i in range(5, 8):
    base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO'
    model_name = base+'_{}_conv'.format(i)
    model = build_2d_cnn_custom_ch_64_DO(conv_num=i)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),
          metrics=['accuracy'])
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    os.makedirs(model_path, exist_ok=True)
    model_filename = model_path+'{epoch:03d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", 
                                   verbose=1, save_best_only=True)

    early_stopping = EarlyStopping(monitor='val_loss', patience=100)
    hist = model.fit_generator(data_generator,
            steps_per_epoch=len(x_train)//batch_size,
            epochs=10000,
            validation_data=(x_val, y_val),
            callbacks = [checkpointer, early_stopping],
            workers=8, 
            use_multiprocessing=True
    )

    print()
    print(model_name, 'Model')
    fig, ax = plt.subplots()
    ax.plot(hist.history['loss'], 'y', label='train loss')
    ax.plot(hist.history['val_loss'], 'r', label='val loss')
    ax.plot(hist.history['acc'], 'b', label='train acc')
    ax.plot(hist.history['val_acc'], 'g', label='val acc')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend(loc='upper left')
    plt.show()

    png_path = 'visualization/learning_curve/'
    filename = model_name+'.png'
    os.makedirs(png_path, exist_ok=True)
    fig.savefig(png_path+filename, transparent=True)

    model.save(model_path+'000_last.hdf5')
    del(model)

    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + sorted(os.listdir(model_path))[-1]
    model = load_model(model_filename)
    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)
    print()

    del(model)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10000
3200/3183 [==============================] - 37s 12ms/sample - loss: 1.2729 - acc: 0.5591

Epoch 00001: val_loss improved from inf to 1.27287, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/001-1.2729.hdf5
242/242 [==============================] - 58s 238ms/step - loss: 1.4174 - acc: 0.4193 - val_loss: 1.2729 - val_acc: 0.5591
Epoch 2/10000
3200/3183 [==============================] - 43s 14ms/sample - loss: 1.1862 - acc: 0.5297

Epoch 00002: val_loss improved from 1.27287 to 1.18617, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/002-1.1862.hdf5
242/242 [==============================] - 78s 322ms/step - loss: 1.1258 - acc: 0.5685 - val_loss: 1.1862 - val_acc: 0.5297
Epoch 3/10000
3200/3183 [==============================] - 39s 12ms/sample - loss: 1.1651 - acc: 0.5447

Epoch 00003: val_lo

3200/3183 [==============================] - 49s 15ms/sample - loss: 0.7074 - acc: 0.7400

Epoch 00014: val_loss improved from 0.73527 to 0.70739, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/014-0.7074.hdf5
242/242 [==============================] - 81s 333ms/step - loss: 0.6352 - acc: 0.7614 - val_loss: 0.7074 - val_acc: 0.7400
Epoch 15/10000
3200/3183 [==============================] - 34s 11ms/sample - loss: 0.8684 - acc: 0.6675

Epoch 00015: val_loss did not improve from 0.70739
242/242 [==============================] - 60s 250ms/step - loss: 0.6193 - acc: 0.7685 - val_loss: 0.8684 - val_acc: 0.6675
Epoch 16/10000
3200/3183 [==============================] - 30s 9ms/sample - loss: 0.4959 - acc: 0.8269

Epoch 00016: val_loss improved from 0.70739 to 0.49586, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/016-0.4959.hdf5
242/242 [================

3200/3183 [==============================] - 34s 11ms/sample - loss: 0.6655 - acc: 0.7556

Epoch 00029: val_loss did not improve from 0.49586
242/242 [==============================] - 66s 274ms/step - loss: 0.4969 - acc: 0.8146 - val_loss: 0.6655 - val_acc: 0.7556
Epoch 30/10000
3200/3183 [==============================] - 41s 13ms/sample - loss: 0.7126 - acc: 0.7525

Epoch 00030: val_loss did not improve from 0.49586
242/242 [==============================] - 71s 295ms/step - loss: 0.4831 - acc: 0.8216 - val_loss: 0.7126 - val_acc: 0.7525
Epoch 31/10000
3200/3183 [==============================] - 34s 11ms/sample - loss: 0.7422 - acc: 0.7250

Epoch 00031: val_loss did not improve from 0.49586
242/242 [==============================] - 62s 257ms/step - loss: 0.4606 - acc: 0.8284 - val_loss: 0.7422 - val_acc: 0.7250
Epoch 32/10000
3200/3183 [==============================] - 43s 14ms/sample - loss: 0.3645 - acc: 0.8797

Epoch 00032: val_loss improved from 0.49586 to 0.36449, saving mod

3200/3183 [==============================] - 41s 13ms/sample - loss: 0.4959 - acc: 0.8400

Epoch 00044: val_loss did not improve from 0.26223
242/242 [==============================] - 67s 276ms/step - loss: 0.4089 - acc: 0.8504 - val_loss: 0.4959 - val_acc: 0.8400
Epoch 45/10000
3200/3183 [==============================] - 38s 12ms/sample - loss: 0.4368 - acc: 0.8481

Epoch 00045: val_loss did not improve from 0.26223
242/242 [==============================] - 66s 272ms/step - loss: 0.4104 - acc: 0.8494 - val_loss: 0.4368 - val_acc: 0.8481
Epoch 46/10000
3200/3183 [==============================] - 40s 13ms/sample - loss: 0.4479 - acc: 0.8528

Epoch 00046: val_loss did not improve from 0.26223
242/242 [==============================] - 60s 249ms/step - loss: 0.3903 - acc: 0.8533 - val_loss: 0.4479 - val_acc: 0.8528
Epoch 47/10000
3200/3183 [==============================] - 38s 12ms/sample - loss: 0.4051 - acc: 0.8650

Epoch 00047: val_loss did not improve from 0.26223
242/242 [======

Epoch 60/10000
3200/3183 [==============================] - 43s 13ms/sample - loss: 0.4810 - acc: 0.8325

Epoch 00060: val_loss did not improve from 0.26223
242/242 [==============================] - 73s 303ms/step - loss: 0.3564 - acc: 0.8729 - val_loss: 0.4810 - val_acc: 0.8325
Epoch 61/10000
3200/3183 [==============================] - 41s 13ms/sample - loss: 0.4365 - acc: 0.8444

Epoch 00061: val_loss did not improve from 0.26223
242/242 [==============================] - 74s 307ms/step - loss: 0.3487 - acc: 0.8697 - val_loss: 0.4365 - val_acc: 0.8444
Epoch 62/10000
3200/3183 [==============================] - 46s 14ms/sample - loss: 0.3318 - acc: 0.8875

Epoch 00062: val_loss did not improve from 0.26223
242/242 [==============================] - 87s 360ms/step - loss: 0.3508 - acc: 0.8745 - val_loss: 0.3318 - val_acc: 0.8875
Epoch 63/10000
3200/3183 [==============================] - 47s 15ms/sample - loss: 0.3699 - acc: 0.8650

Epoch 00063: val_loss did not improve from 0.26223


...

3200/3183 [==============================] - 40s 12ms/sample - loss: 0.4013 - acc: 0.8550

Epoch 00106: val_loss did not improve from 0.25493
242/242 [==============================] - 70s 290ms/step - loss: 0.2682 - acc: 0.9034 - val_loss: 0.4013 - val_acc: 0.8550
Epoch 107/10000
3200/3183 [==============================] - 33s 10ms/sample - loss: 0.3780 - acc: 0.8725

Epoch 00107: val_loss did not improve from 0.25493
242/242 [==============================] - 54s 223ms/step - loss: 0.2660 - acc: 0.9042 - val_loss: 0.3780 - val_acc: 0.8725
Epoch 108/10000
3200/3183 [==============================] - 40s 13ms/sample - loss: 0.3000 - acc: 0.9125

Epoch 00108: val_loss did not improve from 0.25493
242/242 [==============================] - 82s 341ms/step - loss: 0.2673 - acc: 0.9028 - val_loss: 0.3000 - val_acc: 0.9125
Epoch 109/10000
3200/3183 [==============================] - 40s 12ms/sample - loss: 0.2862 - acc: 0.9000

Epoch 00109: val_loss did not improve from 0.25493
242/242 [===

Epoch 122/10000
3200/3183 [==============================] - 42s 13ms/sample - loss: 0.2464 - acc: 0.9284

Epoch 00122: val_loss improved from 0.25493 to 0.24645, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/122-0.2464.hdf5
242/242 [==============================] - 61s 250ms/step - loss: 0.2488 - acc: 0.9106 - val_loss: 0.2464 - val_acc: 0.9284
Epoch 123/10000
3200/3183 [==============================] - 31s 10ms/sample - loss: 0.3874 - acc: 0.8763

Epoch 00123: val_loss did not improve from 0.24645
242/242 [==============================] - 70s 290ms/step - loss: 0.2503 - acc: 0.9096 - val_loss: 0.3874 - val_acc: 0.8763
Epoch 124/10000
3200/3183 [==============================] - 45s 14ms/sample - loss: 0.2362 - acc: 0.9175

Epoch 00124: val_loss improved from 0.24645 to 0.23620, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/124-0.2362.hdf5
242/24

Epoch 137/10000
3200/3183 [==============================] - 35s 11ms/sample - loss: 0.3487 - acc: 0.8894

Epoch 00137: val_loss did not improve from 0.23620
242/242 [==============================] - 91s 374ms/step - loss: 0.2292 - acc: 0.9127 - val_loss: 0.3487 - val_acc: 0.8894
Epoch 138/10000
3200/3183 [==============================] - 36s 11ms/sample - loss: 0.3191 - acc: 0.8800

Epoch 00138: val_loss did not improve from 0.23620
242/242 [==============================] - 84s 348ms/step - loss: 0.2261 - acc: 0.9187 - val_loss: 0.3191 - val_acc: 0.8800
Epoch 139/10000
3200/3183 [==============================] - 43s 13ms/sample - loss: 0.3144 - acc: 0.8950s - loss: 0.3059 - ac - ETA: 2s - loss: 0.3134 - acc: 0

Epoch 00139: val_loss did not improve from 0.23620
242/242 [==============================] - 92s 379ms/step - loss: 0.2386 - acc: 0.9112 - val_loss: 0.3144 - val_acc: 0.8950
Epoch 140/10000
3200/3183 [==============================] - 45s 14ms/sample - loss: 0.3871 - acc: 

Epoch 152/10000
3200/3183 [==============================] - 41s 13ms/sample - loss: 0.3255 - acc: 0.8875

Epoch 00152: val_loss did not improve from 0.23194
242/242 [==============================] - 59s 243ms/step - loss: 0.2293 - acc: 0.9140 - val_loss: 0.3255 - val_acc: 0.8875
Epoch 153/10000
3200/3183 [==============================] - 43s 14ms/sample - loss: 0.3366 - acc: 0.8900

Epoch 00153: val_loss did not improve from 0.23194
242/242 [==============================] - 84s 345ms/step - loss: 0.2114 - acc: 0.9222 - val_loss: 0.3366 - val_acc: 0.8900
Epoch 154/10000
3200/3183 [==============================] - 41s 13ms/sample - loss: 0.2353 - acc: 0.9038

Epoch 00154: val_loss did not improve from 0.23194
242/242 [==============================] - 72s 296ms/step - loss: 0.2182 - acc: 0.9214 - val_loss: 0.2353 - val_acc: 0.9038
Epoch 155/10000
3200/3183 [==============================] - 37s 12ms/sample - loss: 0.3871 - acc: 0.8675

Epoch 00155: val_loss did not improve from 0.23

3200/3183 [==============================] - 49s 15ms/sample - loss: 0.2214 - acc: 0.9200

Epoch 00167: val_loss improved from 0.23119 to 0.22144, saving model to model/checkpoint/vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO_3_conv_checkpoint/167-0.2214.hdf5
242/242 [==============================] - 84s 345ms/step - loss: 0.2135 - acc: 0.9225 - val_loss: 0.2214 - val_acc: 0.9200
Epoch 168/10000
3200/3183 [==============================] - 46s 14ms/sample - loss: 0.3286 - acc: 0.8769

Epoch 00168: val_loss did not improve from 0.22144
242/242 [==============================] - 79s 326ms/step - loss: 0.2169 - acc: 0.9225 - val_loss: 0.3286 - val_acc: 0.8769
Epoch 169/10000
3200/3183 [==============================] - 42s 13ms/sample - loss: 0.4111 - acc: 0.8525

Epoch 00169: val_loss did not improve from 0.22144
242/242 [==============================] - 76s 312ms/step - loss: 0.2135 - acc: 0.9206 - val_loss: 0.4111 - val_acc: 0.8525
Epoch 170/10000
3200/3183 [=======

In [ ]:
log_dir = 'log'
os.makedirs(log_dir, exist_ok=True)
base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_64_DO_075_DO'

with open(path.join(log_dir, base), 'w') as log_file:
    for i in range(5, 8):
        model_name = base+'_{}_conv'.format(i)
        print()
        print(model_name, 'Model')
        model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
        model_filename = model_path + sorted(os.listdir(model_path))[-1]

        model = load_model(model_filename)
        model.summary()

        [loss, accuracy] = model.evaluate(x_test, y_test)
        print('Loss:', loss, 'Accuracy:', accuracy)

        del(model)

        log_file.write('\t'.join([model_name, str(accuracy), str(loss)])+'\n')

In [ ]:
for i in range(5, 8):
    model_name = base+'_{}_conv'.format(i)
    print()
    print(model_name, 'Model')
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + '000_last.hdf5'

    model = load_model(model_filename)
    model.summary()

    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)

    del(model)